<h1>imports</h1>

In [195]:
import pandas as pd
pd.set_option('display.max_columns', 500)
from urllib import request
import json
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder, Normalizer, Binarizer
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

<h1>load data</h1>

In [196]:
data_path = './data/'

In [197]:
train_accounts = pd.read_csv(data_path + 'train_accounts.csv')
# train_users = pd.read_csv(data_path + 'train_users.csv')
# train_events = pd.read_csv(data_path + 'train_events.csv')
# train_subscriptions = pd.read_csv(data_path + 'train_subscriptions.csv')
test_accounts = pd.read_csv(data_path + 'test_accounts.csv')
# test_users = pd.read_csv(data_path + 'test_users.csv')
# test_events = pd.read_csv(data_path + 'test_events.csv')
# test_subscriptions = pd.read_csv(data_path + 'test_subscriptions.csv')

<h1>feature engineering</h1>

look at
churn date
user role
domain

In [198]:
# transform plan_id & utm_cluster_id to str since its categorical
train_accounts['plan_id'] = train_accounts['plan_id'].astype(str)
train_accounts['utm_cluster_id'] = train_accounts['utm_cluster_id'].astype(str)
test_accounts['plan_id'] = test_accounts['plan_id'].astype(str)
test_accounts['utm_cluster_id'] = test_accounts['utm_cluster_id'].astype(str)

In [199]:
def clip_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_in.loc[df_in[col_name] > fence_high, col_name] = fence_high
    df_in.loc[df_in[col_name] < fence_low, col_name] = fence_low
    
def creating_time_features(data):
    time_between_created_trial = pd.to_datetime(data['trial_start']) - pd.to_datetime(data['created_at'])
    time_between_created_subscription = pd.to_datetime(data['subscription_started_at']) - pd.to_datetime(data['created_at'])
    time_between_trial_subscription = pd.to_datetime(data['subscription_started_at']) - pd.to_datetime(data['trial_start'])
    time_between_now_trial = datetime.now() - pd.to_datetime(data['trial_start'])
    time_between_now_subscription = datetime.now() - pd.to_datetime(data['subscription_started_at'])
    time_between_now_created = datetime.now() - pd.to_datetime(data['created_at'])
    time_between_now_churn = datetime.now() - pd.to_datetime(data['churn_date'])
    time_between_churn_subscription = pd.to_datetime(data['churn_date']) - pd.to_datetime(data['subscription_started_at'])

    data = data.assign(created_trial_delta=time_between_created_trial.apply(lambda x: (x.seconds//3600)))
    data = data.assign(created_subscription_delta=time_between_created_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(trial_subscription_delta=time_between_trial_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_trial_delta=time_between_now_trial.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_subscription_delta=time_between_now_subscription.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_created_delta=time_between_now_created.apply(lambda x: (x.seconds//3600)))
    data = data.assign(now_churn_delta=time_between_now_churn.apply(lambda x: (x.seconds//3600)))
    data = data.assign(churn_subscription_delta=time_between_churn_subscription.apply(lambda x: (x.seconds//3600)))
    data['is_subscription'] = (data.subscription_started_at.isna()).astype(int)
    data['is_churn'] = (data.churn_date.isna()).astype(int)
    return data

def creating_size_n_survey_features(data, bins, bins_labels):
    #clip_outlier(data,'company_size')
    data.loc[:,'avg_team_size'] = data[["min_team_size", "max_team_size"]].mean(axis=1)
    data['avg_team_size'].fillna(-1, inplace=True)
    data['avg_team_cat'] = pd.cut(data['avg_team_size'], bins=bins, labels=bins_labels)
    data['avg_team_cat'] = data['avg_team_cat'].astype(str)
    data['survey_answers'] = data[['company_size','max_team_size','min_team_size','user_goal','user_description','team_size']].isna().sum(axis=1)
    data['survey_did_answer'] = data['survey_answers']
    return data

x = pd.concat([train_accounts, test_accounts])
x['country_counts'] = x.groupby('country')['country'].transform('count')
x['region_counts'] = x.groupby('region')['region'].transform('count')

In [200]:
# creating size & survey features
bins = sorted((list(train_accounts["max_team_size"].value_counts().index) + [-1.1, -1, ]))
bins_labels = [str(b) for b in bins[1:]]

train_accounts = creating_time_features(train_accounts)
train_accounts = creating_size_n_survey_features(train_accounts, bins, bins_labels)
test_accounts = creating_time_features(test_accounts)
test_accounts = creating_size_n_survey_features(test_accounts, bins, bins_labels)

<h1>preprocessing</h1>

, 'avg_team_cat'
'now_churn_delta', 'churn_subscription_delta', 'company_size', 'survey_answers'
, 'is_subscription', 'is_churn'
'survey_did_answer'

In [201]:
# We map our features into different types
categorical_features = ['os', 'browser', 'payment_currency', 'device', 'country', 'industry', 'utm_cluster_id',
                         'plan_id', 'avg_team_cat']
normalized_features = ['collection_21_days', 'mrr', 'created_trial_delta', 'created_subscription_delta',
                       'trial_subscription_delta', 'now_trial_delta', 'now_subscription_delta', 'now_created_delta', 
                       'now_churn_delta', 'churn_subscription_delta', 'company_size', 'survey_answers']
normalized_features = []
binary_features = ['survey_did_answer']
untouched_features = ['paying', 'is_subscription', 'is_churn']
KBinsDiscretized_features = []
target = ['lead_score']

# And create a column transformer to handle the manipulation for us
preprocess = make_column_transformer(
    (OneHotEncoder(), categorical_features),
    (Normalizer(), normalized_features),
    (Binarizer(), binary_features)
)

In [202]:
if 'account_id' in train_accounts.columns:
    train_accounts.set_index('account_id', inplace=True)
    test_accounts.set_index('account_id', inplace=True)

# Getting only the relevant features from the dataset
dataset_train = train_accounts[categorical_features + normalized_features + binary_features + untouched_features + target]
dataset_test = test_accounts[categorical_features + normalized_features + binary_features + untouched_features]

# Filling empty values with default values 
def fill_empty_values(dataset):
    dataset.loc[:,categorical_features] = dataset[categorical_features].fillna('')
    dataset.loc[:,normalized_features + binary_features + untouched_features] = dataset[normalized_features + binary_features + untouched_features].fillna(0)
    return dataset

dataset_train = fill_empty_values(dataset_train)
dataset_test = fill_empty_values(dataset_test)

/home/ran/Dev/dh_yyr/venv/lib/python3.6/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [203]:
# Seperating the label
y = dataset_train.pop('lead_score')
# We fit our column transformer on both the train and the test sets
concatenated = pd.concat([dataset_train, dataset_test])
preprocess.fit(concatenated)

# We use transform to finally manipulate the features of our training set
dataset_train = dataset_train[concatenated.columns]
x = preprocess.transform(dataset_train)

<h1>train model</h1>

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05, random_state=42)
model = LogisticRegression(class_weight='balanced', penalty='l1', n_jobs=-1) # 'penalty': ['l1', 'l2'], 'C': [1, 10, 100, 1000]
model.fit(x_train,y_train)
y_pred = model.predict(x_test)

/home/ran/Dev/dh_yyr/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ran/Dev/dh_yyr/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [205]:
print(classification_report(y_test, y_pred, target_names=['not lead','lead']))
print('Acc:  {}'.format(metrics.accuracy_score(y_test, y_pred)))
print('MCC: {}'.format(metrics.matthews_corrcoef(y_test, y_pred)))
print('F1:  {}'.format(metrics.f1_score(y_test, y_pred)))


              precision    recall  f1-score   support

    not lead       0.99      0.79      0.88     66427
        lead       0.09      0.79      0.16      1672

    accuracy                           0.79     68099
   macro avg       0.54      0.79      0.52     68099
weighted avg       0.97      0.79      0.86     68099

Acc:  0.7930366084670847
MCC: 0.21909805310133137
F1:  0.15866762177650431


<h1>submit</h1>

In [206]:
dataset_test = dataset_test[concatenated.columns]
x_submission = preprocess.transform(dataset_test)
y_pred_submission = model.predict(x_submission)
# Creating a dictionary where the keys are the account_ids
# and the values are your predictions
submission_account_ids = [str(int(i))for i in test_accounts.index]
predictions = dict(zip(submission_account_ids, map(int, y_pred_submission)))

In [207]:
group_name = 'fRidaY'

In [210]:
# We validate first that we actually send all the test accounts expected to be sent
if y_pred_submission.shape[0] != 71683 or len(submission_account_ids) != 71683:
  raise Exception("You have to send all of the accounts! Expected: (71683, 71683), Got: ({}, {})".format(y_pred_submission.shape[0], submission_account_ids.shape[0]))

if "group_name" not in vars() or group_name == "":
  group_name = input("Please enter your group's name:")

data = json.dumps({'submitter': group_name, 'predictions': predictions}).encode('utf-8')

req = request.Request("https://leaderboard.datahack.org.il/monday/api/",
                      headers={'Content-Type': 'application/json'},
                      data=data)

res = request.urlopen(req)
print(json.load(res))

{'member': 'fRidaY', 'rank': 2, 'score': 0.22036695149604393}


In [ ]:
set(predictions.values())